<a href="https://colab.research.google.com/github/shakib1wrk/shakib1wrk/blob/main/custom_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pip install tensorflow

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
dataset_path = '/content/drive/MyDrive/Data_Set/Augmented'

# Define input shape and number of classes
input_shape = (512, 512, 3)  # Adjust according to your input data shape
num_classes = 6  # Number of classes (folders)

In [14]:
# Define the CNN model
def create_custom_cnn(input_shape, num_classes):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the output for Dense layers
    model.add(layers.Flatten())

    # Dense layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout layer for regularization
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model


In [15]:
# Create the model
custom_cnn_model = create_custom_cnn(input_shape, num_classes)

# Compile the model
custom_cnn_model.compile(optimizer='adam',
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

# Display the model summary
custom_cnn_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 510, 510, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 255, 255, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 253, 253, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 126, 126, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 124, 124, 128)     73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 62, 62, 128)      

In [16]:
# Set up data generators with data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of data for validation
)


In [19]:
# Set batch size
batch_size = 32

# Create training generator
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Create validation generator
validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Train the model
epochs = 50  # Adjust as needed
history = custom_cnn_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Found 1402 images belonging to 6 classes.
Found 348 images belonging to 6 classes.
Epoch 1/50
43/43 [==============================] - 110s 3s/step - loss: 0.7993 - accuracy: 0.7022 - val_loss: 0.8817 - val_accuracy: 0.6844
Epoch 2/50
43/43 [==============================] - 108s 3s/step - loss: 0.7896 - accuracy: 0.7095 - val_loss: 0.9441 - val_accuracy: 0.6500
Epoch 3/50
43/43 [==============================] - 110s 3s/step - loss: 0.7480 - accuracy: 0.7263 - val_loss: 0.8184 - val_accuracy: 0.6906
Epoch 4/50
43/43 [==============================] - 110s 3s/step - loss: 0.6705 - accuracy: 0.7540 - val_loss: 0.8478 - val_accuracy: 0.7125
Epoch 5/50
43/43 [==============================] - 108s 3s/step - loss: 0.7407 - accuracy: 0.6993 - val_loss: 0.7739 - val_accuracy: 0.7219
Epoch 6/50
43/43 [==============================] - 127s 3s/step - loss: 0.6516 - accuracy: 0.7387 - val_loss: 0.8554 - val_accuracy: 0.7188
Epoch 7/50
43/43 [==============================] - 106s 2s/step - loss

In [20]:
# Save the model
model_name = "RSMZ_4"
custom_cnn_model.save(f'/content/drive/MyDrive/{model_name}.h5')